In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import torchsummary
from torchsummary import summary
import matplotlib.pyplot as plt
from torch.nn.parameter import Parameter
import math
from PIL import Image
import cv2

## data download (MNIST)

In [10]:
transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.05,), (0.5,))
    ])

train_data = datasets.MNIST(root='data/', train=True, transform=transform, download=True)
test_data = datasets.MNIST(root='data/', train=False, transform=transform, download=True)

# data loader
train_loader = DataLoader(dataset = train_data, batch_size=256, shuffle=False)
test_loader = DataLoader(dataset = test_data, batch_size=256, shuffle=False)

In [12]:
class Tanh(nn.Module):
    def forward(self, x):
        return 1.7159*torch.tanh(x*2/3)

In [13]:
class Subsampling(nn.Module):
    def __init__(self, kernel_size, in_channels):
        super(Subsampling, self).__init__()
        self.kernel_size = kernel_size
        self.weight = Parameter(torch.Tensor(1,in_channels,1,1))
        self.bias = Parameter(torch.Tensor(1,in_channels,1,1))
        self.reset_parameters()
    
    def forward(self, x):
        x = F.avg_pool2d(x, self.kernel_size)
        x = x*self.weight+self.bias
        return x
    
    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(1))
        self.bias.data.fill_(0.01)

In [14]:
### modified
class select_conv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(select_conv2d, self).__init__()
        self.weight = Parameter(torch.Tensor(10, in_channels, kernel_size, kernel_size))
        self.bias = Parameter(torch.Tensor(1,out_channels,1,1))
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.reset_parameters()
        
    def map_combine_list(self):
        connection_list = [[0,4,5,6,9,10,11,12,14,15],
                [0,1,5,6,7,10,11,12,13,15],
                [0,1,2,6,7,8,11,13,14,15],
                [1,2,3,6,7,8,9,12,14,15],
                [2,3,4,7,8,9,10,12,13,15],
                [3,4,5,8,9,10,11,13,14,15]]
        return connection_list
    
    def forward(self,x):
        B_size = x.size(0)
        output_size = x.size(3)-self.kernel_size+1
        output = torch.zeros(B_size,self.out_channels,output_size,output_size)
        list_ = self.map_combine_list()
        for i in range(len(list_)):
            output[:,list_[i],:,:] += (F.conv2d(x[:,i,:,:].unsqueeze(1), 
                                    self.weight[:,i,:,:].unsqueeze(1))+self.bias[:,list_[i],:,:])
        return output
    
    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(2))
        self.bias.data.fill_(0.01)

In [15]:
class RBF(nn.Module):
    def __init__(self, in_features, out_features, kernel):
        super(RBF, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.kernel = kernel

    def forward(self, x):
        size = (x.size(0), self.out_features, self.in_features)
        x = x.unsqueeze(1).expand(size)
        c = self.kernel.unsqueeze(0).expand(size)
        output = (x - c).pow(2).sum(-1)
        return output


In [18]:
def rbf_tensor():
    def rbf_kernel(number):
        file = './RBF_kernel/' + str(number)+'_RBF.jpg'
        image = cv2.imread(file, 0)
        output = cv2.threshold(image,127,1,cv2.THRESH_BINARY)[1]*-1+1
        return output
    
    kernel_list = []
    for i in range(10):
        kernel_list.append(rbf_kernel(i).flatten())
    return(torch.Tensor(kernel_list))

kernel = rbf_tensor()

In [20]:
class LeNet5(nn.Module):

    def __init__(self):
        super(LeNet5, self).__init__()

        self.feature_extractor = nn.Sequential(
            # layer C1 
            # 6 feature maps, kernel 5x5  
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            Tanh(),
            # layer S2
            # sub-sampling layer (=Average pool) 2x2, map_size=14
            Subsampling(kernel_size=2, in_channels=6),
            Tanh(),
            # layer C3
            # 16 feature maps, kernel 5x5
            select_conv2d(in_channels=6, out_channels=16, kernel_size=5),
            Tanh(),
            # layer S4
            # sub-sampling layer (=Average pool) 2X2
            Subsampling(kernel_size=2, in_channels=16),
            Tanh(),
            # layer C5
            # 120 feature maps, kernel 5x5
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            Tanh()
        )

        self.classifier = nn.Sequential(
            # layer F6
            nn.Linear(in_features=120, out_features=84),
            Tanh(),
            RBF(in_features=84, out_features=10, kernel=kernel),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [21]:
model=LeNet5()

In [58]:
def train(model, epoch=100, w_decay=0):
    # optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=w_decay)
    
    # loss function
    def loss_fn(output, target):
        loss = output[target==1].pow(2).sum()
        loss += torch.log(np.exp(-0.1)+torch.exp(-output[target==0]).sum())
        return loss

    # training
    train_loss_list=[]
    train_accuracy_list=[]
    for i in range(epoch):
        train_loss = 0
        train_accuracy = 0
        #load data
        for data, target in train_loader:
            data = torch.Tensor(np.pad(data,((0,0),(0,0),(2,2),(2,2)),'constant', constant_values=-0.1))
            target = nn.functional.one_hot(target).float()
            optimizer.zero_grad()

            #forward propagation
            y_pred = model(data)
            loss = loss_fn(y_pred,target)
            
           #backpropagation
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_accuracy += (torch.argmin(model(data),dim=1)==torch.argmax(target,dim=1)).sum().item()


        train_loss /= len(train_loader.dataset)
        train_loss *= 256
        train_loss_list.append(train_loss)

        train_accuracy /= len(train_loader.dataset)
        train_accuracy_list.append(train_accuracy)


        if i%1==0:
            print('epoch : {}/{}, train_loss : {:.8f}, train_acc : {:.2f}'.format(i+1,epoch,train_loss,train_accuracy))
    return(train_loss_list,train_accuracy_list)

In [59]:
train(model,2)

epoch : 1/2, train_loss : 12232.55770417, train_acc : 0.67
epoch : 2/2, train_loss : 3909.58170312, train_acc : 0.92


([12232.557704166667, 3909.581703125],
 [0.6717166666666666, 0.9175666666666666])

In [33]:
summary(model,(1,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              Tanh-2            [-1, 6, 28, 28]               0
       Subsampling-3            [-1, 6, 14, 14]              12
     select_conv2d-4           [-1, 16, 10, 10]           1,516
              Tanh-5           [-1, 16, 10, 10]               0
       Subsampling-6             [-1, 16, 5, 5]              32
            Conv2d-7            [-1, 120, 1, 1]          48,120
              Tanh-8            [-1, 120, 1, 1]               0
            Linear-9                   [-1, 84]          10,164
             Tanh-10                   [-1, 84]               0
              RBF-11                   [-1, 10]               0
Total params: 60,000
Trainable params: 60,000
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [31]:
def test(model):
    accuracy = 0
    for data, target in test_loader:
        data = torch.Tensor(np.pad(data,((0,0),(0,0),(2,2),(2,2)),'constant', constant_values=-0.1))
        y_pred = model(data)
        accuracy += (torch.argmin(y_pred,dim=1)==target).sum().item()
    accuracy /= len(test_loader.dataset)
    return(accuracy)

In [815]:
result = test(model)

In [816]:
result

0.9875

In [822]:
torch.save(model,'LeNet.pt')

In [823]:
model2 = torch.load('LeNet.pt')

In [818]:
model.state_dict('C:\Users\Infosci_center\Desktop\Pytorch_study')

OrderedDict([('feature_extractor.0.weight',
              tensor([[[[-0.1669,  0.0924, -0.2262,  0.0291, -0.1345],
                        [ 1.1318,  0.3818, -0.2767, -0.5473, -0.0067],
                        [ 1.0577, -0.1913, -0.7113, -0.5797,  0.3397],
                        [ 0.0275,  0.0632, -0.1559,  0.3181,  0.5049],
                        [-0.2808,  0.1549,  0.7100,  0.8390,  0.1536]]],
              
              
                      [[[-0.4609, -0.2894, -0.2642, -0.4907, -0.2840],
                        [ 0.3177,  0.1398, -0.0776, -0.2632,  0.9228],
                        [ 1.5891,  1.0968,  1.0943,  1.4901,  0.9536],
                        [-0.1918, -0.0580,  0.4768, -0.1415, -0.1789],
                        [-0.3181, -0.0086, -0.1001, -0.4684, -0.3730]]],
              
              
                      [[[-0.2712,  0.0698,  0.7837,  0.2745,  0.3624],
                        [-0.5300,  0.4713,  1.0665, -0.0494, -0.2926],
                        [-0.4626,  1.085

In [824]:
result2 = test(model2)
print(result2)

0.9875


In [844]:
for param in model2.parameters():
    print(param.shape)

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([1, 6, 1, 1])
torch.Size([1, 6, 1, 1])
torch.Size([16, 6, 5, 5])
torch.Size([1, 16, 1, 1])
torch.Size([1, 16, 1, 1])
torch.Size([1, 16, 1, 1])
torch.Size([120, 16, 5, 5])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
